# 15.1 심층 합성곱 신경망으로 이미지 분류

### 1차원 CNN 구현하기 

In [ ]:
import numpy as np

def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate(
            [zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int((len(x_padded) - len(w_rot)) / s) + 1, s):
        res.append(np.sum(
            x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

In [ ]:
## 테스트
x = [1,3,2,4,5,6,1,3]
w=[1,0,3,1,2]

print('Conv1d 구현:', conv1d(x, w, p=2, s=1))

Conv1d 구현: [ 5. 14. 16. 26. 24. 34. 19. 22.]


In [ ]:
print('넘파이 결과:', np.convolve(x, w, mode='same'))

넘파이 결과: [ 5 14 16 26 24 34 19 22]


### scipy로 2D 합성곱 구현하기

In [ ]:
import numpy as np
import scipy.signal

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1,::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0],
    p[1]:p[1]+X_orig.shape[1]] = X_orig

    res = []
    for i in range(0, int((X_padded.shape[0] - 
                           W_rot.shape[0])/s[0])+1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - 
                               W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0],
                             j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))


In [ ]:
X = [[1,3,2,4],[5,6,1,3],[1,2,0,2],[3,4,3,2]]
W = [[1,0,3],[1,2,1],[0,1,1]]

print('Conv2d 구현:\n', conv2d(X, W, p=(1,1), s=(1,1)))

Conv2d 구현:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]


In [ ]:
print('사이파이 결과 : \n',
      scipy.signal.convolve2d(X, W, mode='same'))

사이파이 결과 : 
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


# 15.2 기본 구성 요소를 사용하여 심층 합성곱 신경망 구성 

In [1]:
from tensorflow import keras
conv_layer = keras.layers.Conv2D(
    filters=16,
    kernel_size=(3,3),
    kernel_regularizer=keras.regularizers.l2(0.001))
fc_layer=keras.layers.Dense(
    units=16,
    kernel_regularizer=keras.regularizers.l2(0.001))

### 분류를 위한 손실 함수


In [6]:
import tensorflow_datasets as tfds
import tensorflow as tf 

### 이진 크로스 엔트로피
bce_probas = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce_logits = tf.keras.losses.BinaryCrossentropy(from_logits=True)
logits = tf.constant([0.8])
probas = tf.keras.activations.sigmoid(logits)
tf.print(
    'BCE (확률) : {:.4f}'.format(bce_probas(y_true=[1], y_pred=probas)),
    'BCE (로짓): {:.4f}'.format(bce_logits(y_true=[1], y_pred=logits))
)


BCE (확률) : 0.3711 BCE (로짓): 0.3711


In [5]:
### 범주형 크로스 엔트로피
cce_probas = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
cce_logits = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
logits = tf.constant([1.5, 0.8, 2.1])
probas = tf.keras.activations.sigmoid(logits)
tf.print(
    'CCE (확률) : {:.4f}'.format(cce_probas(y_true=[0,0,1], y_pred=probas)),
    'CCE (로짓): {:.4f}'.format(cce_logits(y_true=[0,0,1], y_pred=logits))
)

CCE (확률) : 0.5996 CCE (로짓): 0.5996


In [12]:
####### 희소 범주형 크로스 엔트로피
sp_cce_probas = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False)
sp_cce_logits = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True)
logits = tf.constant([1.5, 0.8, 2.1])
probas = tf.keras.activations.sigmoid(logits)
tf.print(
    'Sparse CCE (확률): {:.4f}'.format(
    sp_cce_probas(y_true=[2], y_pred=probas)),
    'Sparse CCE (로짓): {:.4f}'.format(
    sp_cce_logits(y_true=[2], y_pred=logits)))

Sparse CCE (확률): 0.5996 Sparse CCE (로짓): 0.5996
